In [4]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit 
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

def get_unique_list(material_list):
    unique_list = []
    for item in material_list:
        if isinstance(item, str) and "[" in item and "]" in item:
            item_values = ast.literal_eval(item)
            unique_list.extend(item_values)
        else:
            unique_list.append(item)
    return sorted(list(set(unique_list)))

customer_id = '26'
customer_name='%discountschoolsupply%'
client='DSS'

# Curated

In [5]:
# attribut='count'
dateszs='2001-08-11'
# # attribut='features'
# dateszs='2001-11-01'
# curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs}

print('Start Query of Curated Data')
cb = query_from_file(file_name='query/curated_all_attributes_date_family.sql', params=params)
print(len(cb))

fill_dict = cb.groupby('external_id')['buckets'].last().to_dict()
cb['buckets'] = cb['buckets'].fillna(cb['external_id'].map(fill_dict))

Start Query of Curated Data
32283


# Check attribute bucket pairs

In [8]:
print(len(sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())))
lst_attribute=sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())
print('Number of attributes: '+str(len(lst_attribute)))
print('')

# lst_attribute=['color']
for i in range(len(lst_attribute)):
    attri=lst_attribute[i]
    dateszs='2001-08-11'
    attribut=attri
    dateszs='2001-11-01'
    curation_col = f'Q:{attribut}'
    params = {'customer_id': customer_id,
              'customer_name':customer_name,
             'dateszs':dateszs,
             'attribut':attribut}

    print('Start Buckets')
    print(str(i)+' Attribute: '+str(attri))
    bucket_value = query_from_file(file_name='query/Bucket_Value_Strategy.sql', params=params)

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
    print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
#     print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    filtered_dfs = {}
    b_lst=[]

    for i in range(len(list_of_all_buckets)):

        # values that are supposed to be in the buckets
        action_bucket_values=bucket_value[bucket_value['buckets'].astype(str)==list_of_all_buckets[i]]['values'].to_list()

        # values that are actually in the curation
        action=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]['value'].to_list()

        unique_values=get_unique_list(action)

        target=[x for x in unique_values if x not in action_bucket_values]
        if len(target) > 0:
            target = target[0]
            pat=f'''(?i)({target})|()'''
            bucket_filtered_df=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]
            bucket_filtered_df['match']=bucket_filtered_df['value'].apply(lambda x: re_extract(pat,str(x)))
            fil=bucket_filtered_df[bucket_filtered_df['match'].astype(str)!='[]']
            filtered_dfs[list_of_all_buckets[i]] = fil
            b_lst.append(list_of_all_buckets[i])
        else:
            pass

    
    b_lst[0:10]
    valuez = {}
    external_id_list=[]
    for i in range(len(b_lst)):
        valuez[b_lst[i]]=filtered_dfs[b_lst[i]][['external_id','value']]
        external_id_list.append(filtered_dfs[b_lst[i]]['external_id'].tolist())
    import itertools
    flattened_list = list(itertools.chain.from_iterable(external_id_list))
    if len(flattened_list)>0:
        print('Buckets Effected: '+str(len(b_lst)))
        df_name = f'match_{attri}'
        data = {'external_id': flattened_list, f'Q:{attri}': '[]'}
        df_dict = {df_name: pd.DataFrame(data)}
        new_df = df_dict[df_name]
        print('Number of SKUs to be wiped: '+str(len(new_df)))
        print('')
        print('')

        def get_df_name(df):
            name =[x for x in globals() if globals()[x] is df][0]
            return name
        def looks_good(customer, matches,attri): 
            drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
            matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attri}-{today}.csv',index=False) 
        looks_good(client, new_df,attri) 
    else:
        pass

42
Number of attributes: 42

Start Buckets
0 Attribute: capacity_gal
Number of SKUs for the attribute capacity_gal: 2
Start Buckets
1 Attribute: capacity_l
Number of SKUs for the attribute capacity_l: 2
Start Buckets
2 Attribute: capacity_oz
Number of SKUs for the attribute capacity_oz: 1
Start Buckets
3 Attribute: capacity_qt
Number of SKUs for the attribute capacity_qt: 0
Start Buckets
4 Attribute: color
Number of SKUs for the attribute color: 2517
Start Buckets
5 Attribute: count
Number of SKUs for the attribute count: 5780
Start Buckets
6 Attribute: count_per_package
Number of SKUs for the attribute count_per_package: 360
Start Buckets
7 Attribute: count_range
Number of SKUs for the attribute count_range: 5739
Start Buckets
8 Attribute: culture_ethnicity
Number of SKUs for the attribute culture_ethnicity: 20
Start Buckets
9 Attribute: depth
Number of SKUs for the attribute depth: 706
Start Buckets
10 Attribute: diameter
Number of SKUs for the attribute diameter: 28
Start Buckets
11

# Check Freetext Attributes

In [2]:
patterns = {
    'width': r'(\d in)|(\d ft)|(n\/a)|()',
    'volume_oz': r'(\d oz)|(n\/a)|()',
    'volume_lb': r'(\d lb)|(n\/a)|()',
#     'volume_gal':r'(\d gal)|(n\/a)|()',
    'length':r'(\d in)|(\d ft)|(n\/a)|()',
    'diameter':r'(\d in)|(\d ft)|(n\/a)|()',
    'depth':r'(\d in)|(\d ft)|(n\/a)|()',
    'capacity_qt':r'(\d+ qt)|(n\/a)|()',
    'capacity_oz':r'(\d+ oz)|(n\/a)|()',
    'capacity_l':r'(\d+ L)|(n\/a)|()',
    'capacity_gal':r'(\d+ gal)|(n\/a)|()',
#     'count':r'(\d+\,)|(n\/a)|()'
}

In [9]:
# cb[cb['attribute'].astype(str)=='count']['value'].explode().value_counts()[0:500]

In [6]:
def filter_empty_matches(dataframe, attribute, pattern):
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    empty_matches = matches.apply(lambda x: len(x) == 0)
    return dataframe[dataframe['attribute'].astype(str) == attribute][empty_matches]

def count_empty_matches(dataframe, attribute, pattern):
    print(f'Number of SKUs for {attribute}: '+str(len(dataframe[dataframe['attribute'].astype(str) == attribute])))
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    return sum(len(match) == 0 for match in matches)


filtered_dataframes = {}
for attribute, pattern in patterns.items():
    filtered_dataframes[attribute] = filter_empty_matches(cb, attribute, pattern)
    
counts = {}
for attribute, pattern in patterns.items():
    counts[attribute] = count_empty_matches(cb, attribute, pattern)
#     print(f'Rows that need to be wiped for {attribute}: '+f': {counts[attribute]}')
   

    try:
        flattened_list = filtered_dataframes[attribute]['external_id'].to_list()
        if len(flattened_list) > 0:
            df_name = f'match_{attribute}'
            data = {'external_id': flattened_list, f'Q:{attribute}': '[]'}
            df_dict = {df_name: pd.DataFrame(data)}
            new_df = df_dict[df_name]
            print('Number of SKUs to be wiped: ' + str(len(new_df)))
            print('')
            print('')

            def get_df_name(df):
                name = [x for x in globals() if globals()[x] is df][0]
                return name

            def looks_good(customer, matches, attribute):
                today = pd.Timestamp.today().strftime("%Y-%m-%d")
                drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload'
                matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attribute}-{today}.csv', index=False)
            looks_good(client, new_df, attribute)
            
        else:
            print('No SKUs to be wiped for ' + attribute)
    except KeyError:
        print('Error: DataFrame with name ' + df_name + ' does not exist')

Number of SKUs for width: 2587
Number of SKUs to be wiped: 10


Number of SKUs for volume_oz: 1636
Number of SKUs to be wiped: 1


Number of SKUs for volume_lb: 26
No SKUs to be wiped for volume_lb
Number of SKUs for volume_gal: 0
Error: DataFrame with name match_volume_oz does not exist
Number of SKUs for length: 1646
Number of SKUs to be wiped: 2


Number of SKUs for diameter: 112
No SKUs to be wiped for diameter
Number of SKUs for depth: 1303
Number of SKUs to be wiped: 4


Number of SKUs for capacity_qt: 2
No SKUs to be wiped for capacity_qt
Number of SKUs for capacity_oz: 3
No SKUs to be wiped for capacity_oz
Number of SKUs for capacity_l: 2
No SKUs to be wiped for capacity_l
Number of SKUs for capacity_gal: 2
No SKUs to be wiped for capacity_gal
Number of SKUs for count: 5845
Number of SKUs to be wiped: 5777




In [12]:
count_range=cb[cb['attribute'].astype(str)=='count_range']
count=cb[cb['attribute'].astype(str)=='count']

print(len(count_range))
print(len(count))

5799
5845


In [ ]:
count_range['Q:count_range']='[]'
match_count_range=count_range[['external_id','Q:count_range']]

In [ ]:
dateszs='2002-08-11'
attribut='count'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'attribute':attribut}

print('start')
cb = query_from_file(file_name='./query/curated_all_attributes_date_one_attribute.sql', params=params)
print(len(cb))
# dfz=dfs[dfs['resolution'].astype(str)!='rules']
# print(len(dfz))
# dfh=dfz[dfz['custom_fields'].astype(str)!='None']
# # dfz=dfz[dfz['buckets'].astype(str)=='Conduit Fittings, Replacement Parts, & Accessories']
# print(len(dfh))
# dfh=dfs[dfs['custom_fields'].astype(str)!=['None']]
# custom_field_df=pd.json_normalize(dfs['custom_fields'])
# fields=['Set_Size']
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
# # print(len(df))

In [ ]:
count=cb

In [ ]:
pat='''(\d+(?!\d)(?!\-))|(individual)|(Indivdual)|(Individual in)|(Inividual)|()'''
count['match']=count['value'].apply(lambda x: re_extract(pat,str(x)))
c=count[count['match'].astype(str)!='[]']
print(len(c))

In [ ]:
c['Q:count']='[]'
match_count_wipe=c[['external_id','Q:count']]

In [ ]:
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# def looks_good(customer, matches): 
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
#     matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_count_wipe-{today}.csv',index=False) 
# looks_good(client, match_count_wipe) 

# Uncurated

In [2]:
formatted_attribute='count_range'
buckets = """Office Chairs"""

# strategy_version_input=767
# attribute_id_input=2730

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


df = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(df))
print(today)

15
2023_06_03


In [3]:
df_id=df['external_id'].to_list()

In [13]:
x=count[(count['external_id'].isin(df_id))&(count['value'].astype(str)!='Individual')]

In [14]:
print(len(x))
x['value'].explode().value_counts()

1


50    1
Name: value, dtype: int64

In [15]:
x['Q:count']='[]'
match_count=x[['external_id','Q:count']]

In [16]:
pat='''(?i)(.{0,30}(?:single|individual(?!ly wrap)(?!ity)).{0,30})|()'''

In [17]:
df['m_long']=df['long_desc'].apply(lambda x: re_extract(pat,str(x)))
df['m_name']=df['product_name'].apply(lambda x: re_extract(pat,str(x)))
df['m_custom']=df['custom_fields'].apply(lambda x: re_extract(pat,str(x)))

In [18]:
dss=df[(df['m_name'].astype(str)!='[]')|(df['m_long'].astype(str)!='[]')|(df['m_custom'].astype(str)!='[]')]

In [19]:
print(len(dss))
# dss

0


In [20]:
dss_na=df[(df['m_name'].astype(str)=='[]')&(df['m_long'].astype(str)=='[]')&(df['m_custom'].astype(str)=='[]')]
print(len(dss_na))

15


In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_count-{today}.csv',index=False) 
looks_good(client, match_count) 